In [1]:
%load_ext autoreload
%autoreload 2

from jupyter_cadquery import set_defaults, show, Camera, Collapse
from build123d import *
from functools import partial
import math

Hexagon = partial(RegularPolygon, side_count=6)

set_defaults(
    viewer="Sidecar",
    edge_accuracy=0.0001,
    ticks=25,
    grid=[True, True, True],
    cad_width=780,
    axes=True,
    axes0=True,
    glass=True,
    orbit_control=True,
    reset_camera=Camera.CENTER,
    collapse=Collapse.LEAVES
)

Overwriting auto display for build123d BuildPart, BuildSketch, BuildLine, ShapeList


In [34]:
def make_hex_cutout(width, length, margin=0.42, diameter=1):
    corner_radius = diameter / 2
    side_radius = (corner_radius * 3) / math.sqrt(3)
    long_rows = math.floor(width / (diameter + corner_radius))
    short_rows = long_rows + 1
    row_length = math.ceil(length / side_radius)

    locs = GridLocations(corner_radius * 3, side_radius, short_rows, row_length).locations + GridLocations(corner_radius * 3, side_radius, long_rows, row_length + 1).locations
    polygons = ShapeList([loc * Hexagon(corner_radius - margin) for loc in locs])

    return Rectangle(width, length) & polygons

def make_bookmark(svg, mult):
    width = 36
    length = 136
    height = 1.4

    thickness = 4
    gap_width = 2
    cutout_tip = 33
    grate_thickness = 0.42 * 3
    if type(svg) is Face:
        f = scale(svg, mult)
        centered_cutout = (make_hex_cutout(width - thickness * 3, width - thickness * 3, diameter=thickness)) - (Pos(-f.center()) * f)
        cutout = extrude(centered_cutout, height)
    elif svg is not None:
        f = scale(-Face(svg[0]), mult)

        centered_cutout = (Pos(-f.center()) * f)
        cutout = extrude(centered_cutout, height)
    else:
        cutout = extrude(make_hex_cutout(width - thickness * 3, width - thickness * 3, diameter=thickness * 4, margin=grate_thickness), height)
    tongue_cutout = extrude(Pos(0, -cutout_tip * 0.485, 0) * make_hex_cutout(
        width - 2 * (1.75 * thickness + gap_width),
        length - cutout_tip - 2 * (1.75 * thickness + gap_width),
        diameter=thickness * 2,
        margin=grate_thickness
    ), height)
    base_shape = extrude(RectangleRounded(width, length, 2), height) - extrude(Face(
        FilletPolyline(
        (width/2 - (thickness + gap_width), cutout_tip),
        (width/2 - thickness, cutout_tip),
        (width/2 - thickness, -(length / 2 - thickness)),
        (-(width/2 - thickness), -(length / 2 - thickness)),
        (-(width/2 - thickness), cutout_tip),
        (-(width/2 - (thickness + gap_width)), cutout_tip),
        (-(width/2 - (thickness + gap_width)),  -(length / 2 - (thickness + gap_width))),
        ((width/2 - (thickness + gap_width)),  -(length / 2 - (thickness + gap_width))),
        radius=0.999, close=True)
    ), -height) - tongue_cutout
    return base_shape - Pos(0, length / 3 + 5, 0) * cutout

bm = make_bookmark(None, 1)
show(bm)

+


In [35]:
export_stl(bm, "outputs/bookmark.stl")

True